In [54]:
from IPython.display import display, HTML

# CSS to center all DataFrame outputs
css = """
.output {
    display: flex;
    align-items: center;
    justify-content: center;
}
"""

# Apply the CSS
display(HTML('<style>{}</style>'.format(css)))

# Introduction

##### The options market can be a valuable tool in understanding the movements of underlying assets as it allows us to extract information about investors' confidence in them over varying periods. The below process is one such method, allowing us to create features that represent the implied movements and momentum within any given underlying asset.

# Single Contract Implications

##### To start, we must first undertand what a single contract implies about the underlying. To do so, we look at both Calls and Puts that are ITM (Justification for excluding OTM later)

### ITM Call

##### Take for example a $\$11.20$ Call Option with a Strike Price of $\$100$ where the current value of the underlying asset is $\$110$ dollars. Obiously, this Call Option has $\$10$ of instric value since afterall (provided it is American Style) you could just buy the option, exercise it right away then immediately sell to gain $\$10$ on the underlying (obviously you would still lose the remaining value of the contract). However, the speculative premium (found by subtracting the intrinsic value from the total option price) is where we want to focus. In this case, our speculative premium is given by $\$11.20$ - $\$10$ = $\$1.20$. Let's dive deeper into what this truly means:

##### A speculative value of $\$1.20$ implies the underlying is expected to go up $\$1.20$ (In our example, this would result in the underlying sitting at $\$111.20$). If this expectation were any less, the Call Option buyer would not be willing to buy the option (as he would pay $\$11.20$ and expect to make back <$\$11.20$). If the expectation were any higher, then the seller would not be willing to sell the option as while he would collect the $\$11.20$ fee, he would expect to lose more money when the counterparty long the option exercised. Any transact that occurs with these details then implies a $1.20 increase in the asset over the remaining lifetime of the option.

<p align="center">
  <img src="Visuals/CallImplication.png" width="1000" height="750">
</p>

### ITM Put

##### Now, take a $\$11.20$ Put Option with a Strike Price of $\$100$ where the current value of the underlying is $\$90$. This Put Option similarly has $\$10$ of intrinsic value for the same reason of being able to buy the contract and exercise immediately, as well as a $\$1.20$ speculative premium. 

##### For a Put Option, a speculative value of $\$1.20$ implies the underlying will decrease by $\$1.20$ (to $\$88.80$). Again, if this expectation were any greater, the Put Option Buyer would not be willing to transact as he would expect to buy the option for $\$11.20$, yet be unable to recoop this amount by exercising the option at its expiry. Similarly, if this expectation were any less, the Put Option seller would be unable to transact as he would expect to Sell the option, gaining $\$11.20$, but losing more than $\$11.20$ when the counterparty exercised. Once again, see the payoff diagram below:

<p align="center">
  <img src="Visuals/PutImplication.png" width="1000" height="750">
</p>

# Aggregating Single Contract Implications

##### With this understanding of what a single ITM contract implies, we can then aggregate many such contracts to gain information about the overall sentiment regarding the underlying. When aggregating, we think about volume as "votes" or a weighting factor. For example, say we have 1000 total ITM contracts traded on a given day, 500 of which imply a price increase of $1 while the other 500 imply a price decrease of $\$0.99$. To find the aggregate implication we utilize a weighted average:

\begin{array}{c}
\frac{500}{1000} \cdot (1.00) + \frac{500}{1000} \cdot (-0.90) = +0.05
\end{array}

##### This means the ITM options as a whole imply the underlying will increase by 5 cents throughout the duration of the contract. We can perform this process of calculation for any underlying, on any trading day, for a given duration. Below is an example of running such a calculation with the underlying asset being SPY.

In [55]:
import pandas as pd
implications = pd.read_csv("Data/implications.csv")
implications.head(10)

,Date,CurrentPrice,Implied1DayITM,Implied2DayITM,Implied3DayITM,Implied4DayITM,Implied5DayITM,Implied6DayITM,Implied7DayITM,Implied8DayITM,Implied9DayITM
0,10/2/2023,426.140015,0.440886,0.446184,0.412461,0.321686,-0.004224,0.707108,0.745728,0.752794,0.824639
1,10/3/2023,421.200012,-0.189880,-0.134175,-0.364932,-0.801738,-1.294898,-0.807943,-1.425572,-1.209310,0.554315
2,10/4/2023,425.140015,1.051595,1.319109,1.853285,2.369971,2.576255,2.677576,2.238827,3.350653,3.495821
3,10/5/2023,424.536011,0.556481,1.059772,0.583230,1.260467,1.176793,0.811566,1.596727,0.949615,-0.058823
4,10/6/2023,430.459992,0.494715,0.849367,1.294930,1.667953,2.081195,1.932271,2.661532,2.353661,2.188482
5,10/9/2023,431.739990,0.329395,0.747253,0.777285,1.348925,1.642668,1.242887,2.646428,1.606337,1.218661
6,10/10/2023,434.839996,-0.481054,-0.475354,-0.513894,-0.510726,-0.547346,-0.233441,-0.701094,-0.327126,-0.647035
7,10/11/2023,436.059998,1.189536,1.343469,1.782790,1.195499,1.839970,2.664133,1.174898,2.260053,2.810292
8,10/12/2023,433.809998,-0.328892,-0.489099,-0.814340,-0.877466,-0.672780,-0.611394,-0.560701,-1.781753,-1.381419
9,10/13/2023,430.450012,-0.920869,-1.129634,-0.806646,-0.927527,-0.836982,-0.854347,-0.838934,-0.161741,-0.601406


##### With this table, we find that on 10/02/2023, the options market believes the underlying (SPY in this case) will increase by $\$0.44$ over the next day, $\$0.45$ over the next two, $\$0.41$ over the next three, etc...

# Ideas for Application

##### **Capturing Momentum and Sentiment of the Underlying** - running this calculation gives a general idea of what investors think is going to happen to the underlying over a given duration. In addition, shorter term options seem to capture the underlying's momentum.

##### **Calculating Other Implications** - For example, you could calculate the implied Beta between a stock and the market as a whole by using the implied movements in either

##### **Arbitrage** - For example, you would expect that SPY has an identical implication to the result from aggregating and weighting all of S&Ps constituent implications. If there is a disparity (for example the SPY implication is much higher than that of its constituents), on average the speculative premiums for SPY Calls are likely overvalued and the speculative premiums for Constiuent Puts are undervalued. You could potentially exploit this by selling the overvalued and buying the undervalued (by basketing options). This is something I was unable to test due to lack of data. 

# Why Not Use OTM?

##### To wrap up, I wanted to provide a brief justification for not using OTM contracts as part of the calculation. To do so, let's go back to our example of the Call Option with a Strike of $\$100$. If the underlying is instead at $\$90$ and is therefore $\$10$ out of the money, a reasonable price could be $\$1.20$ (Naturally the price is entirely speculative premium). There is two issues, with basing an implication on this: 
- Saying this implies the market will increase by $\$1.20$ means the buyer of the call option will still lose the entire price he paid for the option (as he can just buy for $\$91.20$ instead of the $\$100$ given by the option)
-  OTM prices do not scale linearly as with ITM. The intrinsic value of a Call Option is essentially a ReLU function being at 0 until the underlying hits the Strike, then scaling linearly. Since Options prices are obviously nonnegative, it is unfair to say that a contract $\$5$ OTM is worth $\geq$ $\$5$ more than a contract that is $\$10$ OTM in the way you could say that an option $\$10$ ITM is worth $\geq$ $\$5$ more than a contract only $5 ITM. 
##### The same issues and reasoning can be applied to understanding the shortcomings of OTM Puts as well.